In [1]:
#loading in the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import nibabel as nib
from nltools.data import Brain_Data
from nltools.utils import get_anatomical
from nltools.mask import expand_mask
import seaborn as sns
import sklearn
import pandas as pd

C:\Users\cathe\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
score_data = pd.read_csv('unrestricted_behavData_8_18_2019_21_7_6.csv')

subjects = score_data.Subject.tolist()

Thalamus Data

In [3]:
thalMask = Brain_Data('thalamus_map.nii')

#Each ROI in this parcellation has its own unique number. We can expand this so that each ROI becomes 
#its own binary mask using nltools.mask.expand_mask.
thal_x = expand_mask(thalMask)

#removing the sixth, eighth, and ninth rows (corresponding to ventral medial, ventral posterior medial, and 
#ventral posterior inferior) because they do not have enough voxels
from nltools.mask import collapse_mask
smallThal = thal_x[0, 1, 2, 3, 4, 5, 7, 10, 11, 12]

#We can collapse these 10 separate regions as unique values in a single image
collapseThal = collapse_mask(smallThal)

C:\Users\cathe\Anaconda3\lib\site-packages\nilearn\image\resampling.py:510: UserWarning: Casting data from int16 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))


In [4]:
participant_list_thal = []
roi_corr_list_thal = []
features_list_thal = []
matrix_list_thal = []

#extracting the first 73 subjects; there was an issue where it would not allow for extracting all 100 in one cell
for i in subjects:
    participant_list_thal.append(i)
    
    fname = 'cs4262_hcp/cs4262_hcp/' + str(i) + '/rfMRI_REST1_LR_hp2000_clean.nii.gz'
    brainData = Brain_Data(fname)
    print(i)
    rois = brainData.extract_roi(mask=collapseThal)
    roi_corr = 1 - sklearn.metrics.pairwise_distances(rois, metric='correlation')
    roi_corr_list_thal.append(roi_corr)
    
    features = []
    for i in range(0, roi_corr.shape[0]):
        for j in range (0, i):
            features.append(roi_corr[i,j])
    features_list_thal.append(features)
    
    array = np.asarray(features)
    matrix = np.reshape(array, (1, -1))
    matrix_list_thal.append(matrix)

100307
100408
101107
101309
101915
103111
103414
103818
105014
105115
106016
108828
110411
111312
111716
113619
113922
114419
115320
116524
117122
118528
118730
118932
120111
122317
122620
123117
123925
124422
125525
126325
127630
127933
128127
128632
129028
130013
130316
131217
131722
133019
133928
135225
135932
136833
138534
139637
140925
144832
146432
147737
148335
148840
149337
149539
149741
151223
151526
151627
153025
154734
156637
159340
160123
161731
162733
163129
176542
178950
188347
189450
190031


error: Error -3 while decompressing data: invalid block type

In [7]:
participant_list_thal=participant_list_thal[:73]

In [9]:
#extracting the last 27 subjects
for i in subjects:
    if i not in participant_list_thal:
        participant_list_thal.append(i)
    
        fname = 'cs4262_hcp/cs4262_hcp/' + str(i) + '/rfMRI_REST1_LR_hp2000_clean.nii.gz'
        brainData = Brain_Data(fname)
        print(i)
        rois = brainData.extract_roi(mask=collapseThal)
        roi_corr = 1 - sklearn.metrics.pairwise_distances(rois, metric='correlation')
        roi_corr_list_thal.append(roi_corr)
    
        features = []
        for i in range(0, roi_corr.shape[0]):
            for j in range (0, i):
                features.append(roi_corr[i,j])
        features_list_thal.append(features)
    
        array = np.asarray(features)
        matrix = np.reshape(array, (1, -1))
        matrix_list_thal.append(matrix)

192540
196750
198451
199655
201111
208226
211417
211720
212318
214423
221319
239944
245333
280739
298051
366446
397760
414229
499566
654754
672756
751348
756055
792564
856766
857263
899885


In [10]:
#converting to lists to numpy array
participants = np.array(participant_list_thal)
rois_corrs = np.array(roi_corr_list_thal)
featureses = np.array(features_list_thal)
matrixes = np.array(matrix_list_thal)
print('Done')

Done


In [11]:
#saving the files, specifically cortical_maatrixes, for analysis
np.save('thalmus_participants',participants)
np.save('thalmus_rois_corrs',rois_corrs)
np.save('thalmus_features',featureses)
np.save('thalmus_matrixes',matrixes)
print('Done')

Done
